In [83]:
import os
import numpy as np
import pandas as pd
import cv2
from glob import glob
from tqdm import tqdm

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
path = "dog-breed-identification/"
train_path = os.path.join(path, "train/*")
test_path = os.path.join(path, "test/*")
labels_path = os.path.join(path, "labels.csv")

In [5]:
labels_df = pd.read_csv(labels_path)
# labels_df

In [6]:
breed = labels_df["breed"].unique()
# breed
len(breed)

120

In [81]:
breed2id = {name: i for i, name in enumerate(breed)}
id2breed = {i: name for i, name in enumerate(breed)}

In [8]:
ids = glob(train_path)
# ids

In [12]:
lables = []
for image_id in ids:
    image_id = image_id.split("\\")[-1].split(".")[0]
    # print(image_id)
    breed_name = list(labels_df[labels_df.id == image_id]["breed"])[0]
    # print(image_id, breed_name)
    breed_idx = breed2id[breed_name]
    lables.append(breed_idx)

In [41]:
train_x, valid_x = train_test_split(ids, test_size=0.33, random_state=42)
train_y, valid_y = train_test_split(lables, test_size=0.33, random_state=42)


In [47]:
# Parameters
size = 224
num_classes = 120
lr = 1e-4
batch = 16
epochs = 10

In [48]:
base_model = MobileNetV2(input_shape=(224,224,3), include_top=False, weights="imagenet")

# Freeze the base model
base_model.trainable = False


In [49]:
model = keras.models.Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.2))
model.add(Dense(1024, activation="relu"))
model.add(Dense(num_classes, activation="softmax"))

In [50]:
# model.summary()

In [51]:
model.compile(loss=keras.losses.CategoricalCrossentropy(), optimizer=Adam(learning_rate=lr), metrics=['accuracy'])

In [52]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_2 ( (None, 1280)              0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 1024)              1311744   
_________________________________________________________________
dense_5 (Dense)              (None, 120)               123000    
Total params: 3,692,728
Trainable params: 1,434,744
Non-trainable params: 2,257,984
_________________________________________________________________


In [53]:
def read_image(path, size):
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (224,224))
    image = image/255.0
    image = image.astype(np.float32)
    return image

In [54]:
def parse_data(x,y):
    x = x.decode()
    num_classes = 120
    size = 224
    image =read_image(x,size)
    label = [0] * num_classes
    label[y] = 1
    label = np.array(label)
    label = label.astype(np.int32)
    
    return image, label

In [61]:
def tf_parse(x,y):
    x,y = tf.numpy_function(parse_data, [x,y], [tf.float32, tf.int32])
    x.set_shape((224,224,3))
    y.set_shape((120))
    return x, y

In [74]:
def tf_dataset(x, y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((x,y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.repeat()
    return dataset

In [75]:
# dataset
train_dataset = tf_dataset(train_x, train_y, batch=batch)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch)

In [76]:
# for x,y in valid_dataset:
#     print(x.shape, y.shape)

In [77]:
# Training
callbacks = [
    ModelCheckpoint("model.h5", verbose=1, save_best_only=True),
    ReduceLROnPlateau(factor=0.1, patience=5, min_lr=1e-6)
]

train_steps = (len(train_x)//batch) + 1
valid_steps = (len(valid_x)//batch) + 1

In [78]:
model.fit(train_dataset, steps_per_epoch=train_steps, validation_data=valid_dataset,
          validation_steps=valid_steps, epochs=epochs, callbacks=callbacks)

Epoch 1/10
429/429 [==============================] - 28s 65ms/step - loss: 0.7482 - accuracy: 0.7912 - val_loss: 1.3249 - val_accuracy: 0.6108

Epoch 00001: val_loss improved from inf to 1.32490, saving model to model.h5
Epoch 2/10
  1/429 [..............................] - ETA: 13s - loss: 0.6840 - accuracy: 0.9375

C:\Users\Mehul\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


429/429 [==============================] - 28s 66ms/step - loss: 0.6288 - accuracy: 0.8341 - val_loss: 1.3283 - val_accuracy: 0.6194

Epoch 00002: val_loss did not improve from 1.32490
Epoch 3/10
429/429 [==============================] - 28s 66ms/step - loss: 0.5383 - accuracy: 0.8601 - val_loss: 1.3365 - val_accuracy: 0.6165

Epoch 00003: val_loss did not improve from 1.32490
Epoch 4/10
429/429 [==============================] - 29s 67ms/step - loss: 0.4625 - accuracy: 0.8830 - val_loss: 1.3445 - val_accuracy: 0.6203

Epoch 00004: val_loss did not improve from 1.32490
Epoch 5/10
429/429 [==============================] - 29s 67ms/step - loss: 0.4031 - accuracy: 0.9027 - val_loss: 1.3588 - val_accuracy: 0.6183

Epoch 00005: val_loss did not improve from 1.32490
Epoch 6/10
429/429 [==============================] - 29s 67ms/step - loss: 0.3483 - accuracy: 0.9229 - val_loss: 1.3548 - val_accuracy: 0.6286

Epoch 00006: val_loss did not improve from 1.32490
Epoch 7/10
429/429 [===========

In [79]:
# testing

In [80]:
model_test = keras.models.load_model("model.h5")

In [91]:
for i,path in tqdm(enumerate(valid_x[:10])):
    image = read_image(path, 224)
    image = np.expand_dims(image, axis=0)
    pred = model_test.predict(image)[0]
    label_idx = np.argmax(pred)
    breed_name = id2breed[label_idx]
    
    ori_breed = id2breed[valid_y[i]]
    ori_image = cv2.imread(path, cv2.IMREAD_COLOR)
    ori_image = cv2.putText(ori_image, breed_name, (0,10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (225,0,0), 1)
    ori_image = cv2.putText(ori_image, ori_breed, (0,30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,225,225), 1)
    cv2.imwrite(f"save/valid_{i}.png", ori_image)

10it [00:00, 14.17it/s]
